# Día 2: Programación Funcional en Python

## Descripción General

La programación funcional es un paradigma que trata la computación como la evaluación de funciones matemáticas, evitando el cambio de estado y los datos mutables. Python, aunque es principalmente un lenguaje multiparadigma, proporciona herramientas poderosas para la programación funcional como `map()`, `filter()`, `reduce()`, y el módulo `functools`.

En este notebook, exploraremos las funciones de orden superior más importantes de Python y aprenderemos a usar `functools` para crear código más eficiente y expresivo. Estas técnicas son especialmente útiles en procesamiento de datos, pipelines de transformación, y optimización de rendimiento.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Utilizar `map()`, `filter()` y `reduce()` para transformar y procesar colecciones de datos
2. Aplicar `functools.partial()` para crear funciones especializadas
3. Optimizar funciones con `functools.lru_cache()` para mejorar el rendimiento
4. Combinar técnicas funcionales para crear pipelines de procesamiento de datos
5. Decidir cuándo usar programación funcional vs comprehensions o loops tradicionales

## 1. map() - Transformar Colecciones

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Estás procesando un dataset de 1 millón de registros de sensores IoT. Cada registro tiene temperatura en Celsius, pero tu modelo de ML requiere Fahrenheit. Necesitas transformar todos los valores rápidamente sin escribir loops explícitos.

**Ejemplo concreto para juniors**:

Tienes una lista de 10,000 precios en formato string (`["10.50", "25.99", ...]`). Necesitas:
1. Convertir cada string a float
2. Aplicar conversión de moneda (USD → EUR)
3. Formatear para display

Sin `map()`: 5-10 líneas de código con loops explícitos.
Con `map()`: 1 línea expresiva y clara.

**Consecuencias de NO usarlo**:
- **Código más verboso** → más difícil de mantener
- **Menos expresivo** → intención menos clara
- **No aprovecha lazy evaluation** → consume más memoria con datasets grandes
- **Menos funcional** → dificulta composición de transformaciones

### 📚 El Concepto

**Definición técnica**:

`map(function, iterable)` es una función de orden superior que aplica una función a cada elemento de un iterable y devuelve un iterador con los resultados transformados.

**Cómo funciona internamente**:
1. Python crea un objeto map (iterador lazy)
2. NO ejecuta la función inmediatamente
3. Cuando consumes el iterador (list(), for, next()):
   - Toma siguiente elemento del iterable
   - Aplica la función
   - Yield el resultado
4. Repite hasta agotar el iterable

**Terminología clave**:
- **Función de orden superior**: Función que toma otra función como argumento
- **Lazy evaluation**: No calcula hasta que se necesita
- **Mapping**: Transformación uno-a-uno de elementos
- **Iterator**: Objeto que produce valores bajo demanda

### ❌ MAL: Loop Explícito

In [ ]:
# Transform list of numbers to their squares
numbers = [1, 2, 3, 4, 5]
squares = []
for num in numbers:
    squares.append(num ** 2)
print(squares)

### ✅ BIEN: Usando map()

In [ ]:
# Using map with a lambda function
numbers = [1, 2, 3, 4, 5]
squares = list(map(lambda x: x ** 2, numbers))
print(squares)

# Using map with a named function
def square(x: int) -> int:
    """Calculate the square of a number."""
    return x ** 2

squares = list(map(square, numbers))
print(squares)

# Map with multiple iterables
numbers1 = [1, 2, 3]
numbers2 = [10, 20, 30]
sums = list(map(lambda x, y: x + y, numbers1, numbers2))
print(f"Sums: {sums}")  # [11, 22, 33]

### Ejemplo Práctico: Procesamiento de Datos

In [ ]:
# Convert temperatures from Celsius to Fahrenheit
celsius_temps = [0, 10, 20, 30, 40]

def celsius_to_fahrenheit(celsius: float) -> float:
    """
    Convert Celsius to Fahrenheit.
    
    :param celsius: Temperature in Celsius
    :type celsius: float
    :return: Temperature in Fahrenheit
    :rtype: float
    """
    return (celsius * 9/5) + 32

fahrenheit_temps = list(map(celsius_to_fahrenheit, celsius_temps))
print(f"Celsius: {celsius_temps}")
print(f"Fahrenheit: {fahrenheit_temps}")

# Parse string data
string_numbers = ["1", "2", "3", "4", "5"]
int_numbers = list(map(int, string_numbers))
print(f"Parsed integers: {int_numbers}")

# Extract attributes from objects
class Person:
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

people = [Person("Alice", 30), Person("Bob", 25), Person("Charlie", 35)]
names = list(map(lambda p: p.name, people))
print(f"Names: {names}")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. `map()` retorna un **iterador lazy** (no lista) → consume bajo demanda
2. Puedes mapear **múltiples iterables** → `map(func, iter1, iter2)`
3. Se detiene en el iterable **más corto**
4. Para casos simples, **list comprehensions son más Pythonic**

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Estoy aplicando la MISMA transformación a TODOS los elementos?"
  - SÍ + función ya existe → `map()` es ideal
  - SÍ + transformación simple → list comprehension
  - NO (lógica condicional compleja) → loop tradicional

- **Pregúntate**: "¿Necesito lazy evaluation?"
  - SÍ (dataset grande) → `map()` sin convertir a lista
  - NO (dataset pequeño) → list comprehension

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Tienes función existente para aplicar (`int`, `str.upper`, custom function)
  - Necesitas lazy evaluation (datasets grandes)
  - Mapeas múltiples iterables en paralelo
  - Código funcional/pipeline de transformaciones
  
- ❌ **NO usar cuando**:
  - Transformación simple → list comprehension más legible
  - Necesitas filtrado también → list comprehension con `if`
  - Lambda compleja → mejor función nombrada o loop
  - Efectos secundarios (print, modificar estado)

**Referencia oficial:** [Python map() built-in function](https://docs.python.org/3/library/functions.html#map)

## 2. filter() - Filtrar Colecciones

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Estás construyendo un sistema de recomendación. Tienes 1 millón de productos, pero solo quieres recomendar productos:
- En stock (inventory > 0)
- Con rating > 4.0
- En categoría específica

Necesitas filtrar eficientemente sin cargar todo en memoria.

**Ejemplo concreto para juniors**:

Tienes lista de 10,000 usuarios. Necesitas:
1. Filtrar solo usuarios activos
2. Filtrar solo mayores de 18
3. Filtrar por país específico

Sin `filter()`: Múltiples loops anidados, difícil de leer.
Con `filter()`: Pipeline claro y componible.

**Consecuencias de NO usarlo**:
- **Código menos declarativo** → intención menos clara
- **Dificulta composición** → no puedes encadenar filtros fácilmente
- **Menos funcional** → no aprovechas paradigma funcional
- **Más verboso** para filtros simples con funciones existentes

### 📚 El Concepto

**Definición técnica**:

`filter(function, iterable)` es una función de orden superior que aplica una función predicado (que retorna True/False) a cada elemento y devuelve un iterador con solo los elementos que pasan el test.

**Cómo funciona internamente**:
1. Python crea objeto filter (iterador lazy)
2. Cuando consumes el iterador:
   - Toma siguiente elemento
   - Aplica función predicado
   - Si True → yield elemento
   - Si False → skip, continúa con siguiente
3. Repite hasta agotar iterable

**Terminología clave**:
- **Predicado**: Función que retorna True/False
- **Filtering**: Selección de elementos que cumplen condición
- **Truthy/Falsy**: Si function=None, filtra valores falsy
- **Lazy iterator**: No evalúa hasta consumir

### ❌ MAL: Loop con Condición

In [ ]:
# Filter even numbers
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
evens = []
for num in numbers:
    if num % 2 == 0:
        evens.append(num)
print(evens)

### ✅ BIEN: Usando filter()

In [ ]:
# Using filter with lambda
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
evens = list(filter(lambda x: x % 2 == 0, numbers))
print(f"Even numbers: {evens}")

# Using filter with named function
def is_even(x: int) -> bool:
    """Check if a number is even."""
    return x % 2 == 0

evens = list(filter(is_even, numbers))
print(f"Even numbers: {evens}")

# Filter with None removes falsy values
mixed_data = [0, 1, False, True, "", "hello", None, [], [1, 2]]
truthy_values = list(filter(None, mixed_data))
print(f"Truthy values: {truthy_values}")

### Ejemplo Práctico: Filtrado de Datos

In [ ]:
# Filter users by age
class User:
    def __init__(self, name: str, age: int, active: bool):
        self.name = name
        self.age = age
        self.active = active
    
    def __repr__(self):
        return f"User({self.name}, {self.age}, active={self.active})"

users = [
    User("Alice", 25, True),
    User("Bob", 17, True),
    User("Charlie", 30, False),
    User("David", 22, True),
    User("Eve", 16, True)
]

# Filter adult active users
def is_adult_active(user: User) -> bool:
    """
    Check if user is adult and active.
    
    :param user: User object
    :type user: User
    :return: True if user is 18+ and active
    :rtype: bool
    """
    return user.age >= 18 and user.active

adult_active_users = list(filter(is_adult_active, users))
print(f"Adult active users: {adult_active_users}")

# Filter valid email addresses
emails = ["user@example.com", "invalid", "test@test.com", "", "admin@site.org"]
valid_emails = list(filter(lambda e: "@" in e and "." in e, emails))
print(f"Valid emails: {valid_emails}")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. `filter()` retorna **iterador lazy** → eficiente en memoria
2. `filter(None, iterable)` **filtra valores falsy** (0, "", None, False, [])
3. Función debe retornar **True/False** (o truthy/falsy)
4. Para casos simples, **list comprehension con `if` es más Pythonic**

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Estoy SELECCIONANDO elementos que cumplen condición?"
  - SÍ + función predicado ya existe → `filter()` es ideal
  - SÍ + condición simple → list comprehension con `if`
  - NO (transformando, no filtrando) → `map()`

- **Pregúntate**: "¿Necesito encadenar múltiples filtros?"
  - SÍ → `filter()` permite composición clara
  - NO → list comprehension puede ser más legible

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Tienes función predicado existente
  - Necesitas lazy evaluation (datasets grandes)
  - Encadenas múltiples filtros
  - Código funcional/pipeline
  - Quieres filtrar valores falsy (`filter(None, ...)`)
  
- ❌ **NO usar cuando**:
  - Condición simple → list comprehension más legible: `[x for x in items if x > 0]`
  - Necesitas transformar Y filtrar → list comprehension
  - Lambda compleja → mejor función nombrada o loop
  - Dataset pequeño y necesitas lista → comprehension directa

**Referencia oficial:** [Python filter() built-in function](https://docs.python.org/3/library/functions.html#filter)

## 3. reduce() - Reducir Colecciones a un Valor

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Estás procesando logs de un sistema distribuido. Tienes 1 millón de eventos, cada uno con métricas (latencia, errores, throughput). Necesitas:
- Calcular latencia total acumulada
- Encontrar el evento con mayor throughput
- Combinar múltiples diccionarios de métricas

Todas estas son operaciones de **reducción**: combinar muchos valores en uno.

**Ejemplo concreto para juniors**:

Tienes lista de carritos de compra (cada uno es lista de productos). Necesitas:
1. Calcular precio total de TODOS los productos
2. Encontrar el producto más caro
3. Combinar todos los carritos en uno solo

Sin `reduce()`: Loop con variable acumuladora, propenso a errores.
Con `reduce()`: Operación de reducción clara y expresiva.

**Consecuencias de NO usarlo**:
- **Menos expresivo** → intención menos clara ("estoy reduciendo")
- **Más propenso a errores** → olvidas inicializar acumulador
- **No componible** → difícil de usar en pipelines funcionales
- **Menos declarativo** → describes CÓMO en lugar de QUÉ

### 📚 El Concepto

**Definición técnica**:

`reduce(function, iterable, initial)` aplica una función de dos argumentos acumulativamente a los elementos del iterable, de izquierda a derecha, para reducirlo a un único valor.

**Cómo funciona internamente**:
1. Toma primer elemento (o `initial` si se proporciona) como acumulador
2. Toma siguiente elemento del iterable
3. Aplica función: `acumulador = function(acumulador, elemento)`
4. Repite paso 2-3 hasta agotar iterable
5. Retorna acumulador final

**Ejemplo visual**:
```
reduce(lambda x, y: x + y, [1, 2, 3, 4], 0)

Paso 1: acc = 0 (initial)
Paso 2: acc = func(0, 1) = 1
Paso 3: acc = func(1, 2) = 3
Paso 4: acc = func(3, 3) = 6
Paso 5: acc = func(6, 4) = 10
Resultado: 10
```

**Terminología clave**:
- **Reducción**: Combinar múltiples valores en uno
- **Acumulador**: Valor que se va construyendo
- **Función binaria**: Toma 2 argumentos (acumulador, elemento)
- **Initial value**: Valor inicial del acumulador (opcional pero recomendado)

### ❌ MAL: Loop con Acumulador

In [ ]:
# Calculate product of all numbers
numbers = [1, 2, 3, 4, 5]
product = 1
for num in numbers:
    product *= num
print(f"Product: {product}")

### ✅ BIEN: Usando reduce()

In [ ]:
from functools import reduce

# Calculate product using reduce
numbers = [1, 2, 3, 4, 5]
product = reduce(lambda x, y: x * y, numbers)
print(f"Product: {product}")

# With initial value
product = reduce(lambda x, y: x * y, numbers, 1)
print(f"Product with initial: {product}")

# Sum (though sum() is better for this)
total = reduce(lambda x, y: x + y, numbers)
print(f"Sum: {total}")

# Find maximum
maximum = reduce(lambda x, y: x if x > y else y, numbers)
print(f"Maximum: {maximum}")

### Ejemplo Práctico: Operaciones Complejas

In [ ]:
from functools import reduce

# Flatten nested lists
nested_lists = [[1, 2], [3, 4], [5, 6]]
flattened = reduce(lambda acc, lst: acc + lst, nested_lists, [])
print(f"Flattened: {flattened}")

# Merge dictionaries
dicts = [{"a": 1}, {"b": 2}, {"c": 3}]
merged = reduce(lambda acc, d: {**acc, **d}, dicts, {})
print(f"Merged dict: {merged}")

# Calculate factorial
def factorial(n: int) -> int:
    """
    Calculate factorial using reduce.
    
    :param n: Number to calculate factorial for
    :type n: int
    :return: Factorial of n
    :rtype: int
    """
    if n == 0:
        return 1
    return reduce(lambda x, y: x * y, range(1, n + 1))

print(f"5! = {factorial(5)}")
print(f"10! = {factorial(10)}")

# Compose functions
def compose(*functions):
    """
    Compose multiple functions into one.
    
    :param functions: Functions to compose
    :return: Composed function
    :rtype: callable
    """
    return reduce(lambda f, g: lambda x: f(g(x)), functions)

# Create a pipeline: add 1, multiply by 2, subtract 3
add_one = lambda x: x + 1
multiply_two = lambda x: x * 2
subtract_three = lambda x: x - 3

pipeline = compose(subtract_three, multiply_two, add_one)
result = pipeline(5)  # ((5 + 1) * 2) - 3 = 9
print(f"Pipeline result: {result}")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. `reduce()` está en **functools**, no es built-in (Guido lo considera menos legible)
2. **SIEMPRE proporciona `initial`** → evita errores con iterables vacíos
3. Función debe tomar **2 argumentos**: (acumulador, elemento_actual)
4. Para operaciones comunes, usa **built-ins**: `sum()`, `max()`, `min()`, `any()`, `all()`

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Estoy COMBINANDO todos los elementos en UN solo valor?"
  - SÍ + operación simple (suma, max) → usa built-in (`sum()`, `max()`)
  - SÍ + operación compleja (merge dicts, flatten) → `reduce()` es ideal
  - NO (transformando o filtrando) → `map()` o `filter()`

- **Pregúntate**: "¿Existe un built-in para esto?"
  - SÍ → usa el built-in (más legible)
  - NO → `reduce()` es apropiado

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Operación de reducción compleja (no hay built-in)
  - Flatten nested structures
  - Merge múltiples dicts/sets
  - Composición de funciones
  - Implementar fold/scan patterns
  
- ❌ **NO usar cuando**:
  - Existe built-in: `sum()` en lugar de `reduce(lambda x,y: x+y, ...)`
  - Existe built-in: `max()` en lugar de `reduce(lambda x,y: x if x>y else y, ...)`
  - Operación simple → loop es más legible
  - Lógica compleja → función nombrada con loop

**Alternativas más Pythonic**:
```python
# ❌ NO: reduce para suma
reduce(lambda x, y: x + y, numbers)

# ✅ SÍ: built-in sum
sum(numbers)

# ❌ NO: reduce para max
reduce(lambda x, y: x if x > y else y, numbers)

# ✅ SÍ: built-in max
max(numbers)
```

**Referencia oficial:** [functools.reduce()](https://docs.python.org/3/library/functools.html#functools.reduce)

## 4. functools.partial() - Aplicación Parcial de Funciones

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Estás construyendo un pipeline de preprocesamiento de datos. Tienes una función genérica `normalize_data(data, method, scale, center)` pero necesitas:
- `normalize_standard`: method="standard", scale=True, center=True
- `normalize_minmax`: method="minmax", scale=True, center=False
- `normalize_robust`: method="robust", scale=True, center=True

Sin `partial()`: Crear 3 funciones wrapper manualmente (código duplicado).
Con `partial()`: Crear 3 funciones especializadas en 3 líneas.

**Ejemplo concreto para juniors**:

Tienes función `send_email(to, subject, body, priority, cc)`. Necesitas:
- `send_alert`: priority="high", subject="ALERT"
- `send_notification`: priority="low", subject="Notification"
- `send_to_admin`: to="admin@company.com", priority="high"

**Consecuencias de NO usarlo**:
- **Código duplicado** → más difícil de mantener
- **Funciones wrapper verbosas** → más líneas de código
- **Menos reutilizable** → no puedes pasar funciones especializadas fácilmente
- **Lambdas complejas** → difíciles de leer y debuggear

### 📚 El Concepto

**Definición técnica**:

`partial(func, *args, **kwargs)` crea una nueva función con algunos argumentos "congelados" (pre-filled). Es una técnica de programación funcional llamada "aplicación parcial" o "currying parcial".

**Cómo funciona internamente**:
1. `partial()` crea un objeto callable que envuelve la función original
2. Guarda los argumentos fijos en el objeto
3. Cuando llamas a la función parcial:
   - Combina argumentos fijos + nuevos argumentos
   - Llama a la función original con todos los argumentos
4. Retorna el resultado

**Ejemplo visual**:
```python
def power(base, exponent):
    return base ** exponent

square = partial(power, exponent=2)
# square(5) → power(5, exponent=2) → 25
```

**Terminología clave**:
- **Aplicación parcial**: Fijar algunos argumentos de una función
- **Currying**: Transformar función de N args en N funciones de 1 arg
- **Función de orden superior**: Función que retorna otra función
- **Closure**: Función que captura variables de su entorno

### ❌ MAL: Funciones Wrapper Manuales

In [ ]:
def power(base: float, exponent: float) -> float:
    """Calculate base raised to exponent."""
    return base ** exponent

# Creating specialized functions manually
def square(x: float) -> float:
    """Calculate square of x."""
    return power(x, 2)

def cube(x: float) -> float:
    """Calculate cube of x."""
    return power(x, 3)

print(square(5))
print(cube(3))

### ✅ BIEN: Usando partial()

In [ ]:
from functools import partial

def power(base: float, exponent: float) -> float:
    """Calculate base raised to exponent."""
    return base ** exponent

# Create specialized functions using partial
square = partial(power, exponent=2)
cube = partial(power, exponent=3)

print(f"5 squared: {square(5)}")
print(f"3 cubed: {cube(3)}")

# Partial with positional arguments
def multiply(x: float, y: float, z: float) -> float:
    """Multiply three numbers."""
    return x * y * z

double = partial(multiply, 2)  # Fix first argument to 2
print(f"Double of 5 * 3: {double(5, 3)}")  # 2 * 5 * 3 = 30

### Ejemplo Práctico: Configuración de Funciones

In [ ]:
from functools import partial
import json

# Create specialized JSON dumpers
compact_json = partial(json.dumps, separators=(',', ':'))
pretty_json = partial(json.dumps, indent=4, sort_keys=True)

data = {"name": "Alice", "age": 30, "city": "NYC"}
print("Compact:")
print(compact_json(data))
print("\nPretty:")
print(pretty_json(data))

# Create specialized logging functions
def log_message(message: str, level: str = "INFO", prefix: str = "") -> None:
    """
    Log a message with level and prefix.
    
    :param message: Message to log
    :type message: str
    :param level: Log level
    :type level: str
    :param prefix: Prefix for the message
    :type prefix: str
    """
    print(f"[{level}] {prefix}{message}")

error_log = partial(log_message, level="ERROR", prefix="❌ ")
warning_log = partial(log_message, level="WARNING", prefix="⚠️  ")
info_log = partial(log_message, level="INFO", prefix="ℹ️  ")

error_log("Database connection failed")
warning_log("Deprecated function used")
info_log("Application started successfully")

# Partial with map for data transformation
def convert_temperature(temp: float, from_unit: str, to_unit: str) -> float:
    """
    Convert temperature between units.
    
    :param temp: Temperature value
    :type temp: float
    :param from_unit: Source unit (C or F)
    :type from_unit: str
    :param to_unit: Target unit (C or F)
    :type to_unit: str
    :return: Converted temperature
    :rtype: float
    """
    if from_unit == "C" and to_unit == "F":
        return (temp * 9/5) + 32
    elif from_unit == "F" and to_unit == "C":
        return (temp - 32) * 5/9
    return temp

celsius_to_fahrenheit = partial(convert_temperature, from_unit="C", to_unit="F")
celsius_temps = [0, 10, 20, 30, 40]
fahrenheit_temps = list(map(celsius_to_fahrenheit, celsius_temps))
print(f"\nCelsius: {celsius_temps}")
print(f"Fahrenheit: {fahrenheit_temps}")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. `partial()` **NO ejecuta** la función, crea nueva función con args fijos
2. Puedes fijar **args posicionales Y keyword args**
3. Args nuevos se **añaden** a los fijos (no los reemplazan)
4. Útil para **configurar callbacks** y **crear funciones especializadas**

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Estoy llamando la MISMA función con ALGUNOS args siempre iguales?"
  - SÍ + args se repiten mucho → `partial()` elimina repetición
  - SÍ + necesitas pasar función a otra función → `partial()` es ideal
  - NO (args siempre diferentes) → no necesitas partial

- **Pregúntate**: "¿Necesito crear múltiples versiones especializadas?"
  - SÍ → `partial()` es perfecto (ej: loggers, normalizers, validators)
  - NO → llamada directa está bien

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Crear funciones especializadas (ej: `square = partial(power, exp=2)`)
  - Configurar callbacks con parámetros fijos
  - Adaptar firmas de funciones para APIs
  - Usar con `map()`, `filter()` para fijar args
  - Crear familias de funciones relacionadas
  
- ❌ **NO usar cuando**:
  - Solo llamas la función 1-2 veces → llamada directa
  - Lambda simple es más clara: `lambda x: func(x, 5)`
  - Necesitas lógica condicional → función completa
  - Args cambian frecuentemente → no hay beneficio

**Comparación con alternativas**:
```python
# Opción 1: partial (mejor para reutilización)
double = partial(multiply, 2)
list(map(double, numbers))

# Opción 2: lambda (mejor para uso único)
list(map(lambda x: multiply(2, x), numbers))

# Opción 3: función wrapper (mejor para lógica compleja)
def double(x):
    return multiply(2, x)
```

**Referencia oficial:** [functools.partial()](https://docs.python.org/3/library/functools.html#functools.partial)

## 5. functools.lru_cache() - Memoización para Optimización

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Estás construyendo un sistema de recomendación. Calculas similitud entre usuarios usando una función costosa que:
- Carga perfil del usuario desde DB (100ms)
- Calcula embeddings (50ms)
- Compara con otros usuarios (200ms)

Total: **350ms por usuario**. Con 1000 usuarios y múltiples requests, calculas lo mismo miles de veces.

Sin caché: 1000 usuarios × 350ms = **350 segundos** (5.8 minutos) 💥
Con `lru_cache`: Primera vez 350ms, siguientes **0.001ms** ✅

**Ejemplo concreto para juniors**:

Tienes función que calcula Fibonacci recursivamente. Para `fib(35)`:
- Sin caché: **9,227,465 llamadas recursivas**, 3-5 segundos
- Con caché: **35 llamadas**, 0.0001 segundos

**Consecuencias de NO usarlo**:
- **Cálculos repetidos** → desperdicio de CPU
- **Respuestas lentas** → mala experiencia de usuario
- **No escala** → colapsa con más usuarios/datos
- **Costos más altos** → necesitas más servidores

### 📚 El Concepto

**Definición técnica**:

`@lru_cache(maxsize=128)` es un decorador que implementa memoización con política LRU (Least Recently Used). Cachea resultados de llamadas a funciones basándose en sus argumentos.

**Cómo funciona internamente**:
1. Primera llamada `func(args)`:
   - Ejecuta función
   - Guarda resultado en dict: `{args: resultado}`
   - Retorna resultado
2. Siguientes llamadas con mismos args:
   - Busca en caché
   - Si existe → retorna inmediatamente (sin ejecutar)
   - Si no existe → ejecuta y cachea
3. Cuando caché llena (maxsize alcanzado):
   - Elimina entrada menos recientemente usada (LRU)
   - Añade nueva entrada

**Terminología clave**:
- **Memoización**: Caché de resultados de funciones
- **LRU**: Least Recently Used (elimina lo menos usado)
- **Cache hit**: Resultado encontrado en caché
- **Cache miss**: Resultado NO en caché, debe calcular
- **maxsize**: Tamaño máximo del caché (None = ilimitado)

**Requisitos para usar lru_cache**:
1. Función debe ser **pura** (sin efectos secundarios)
2. Argumentos deben ser **hashables** (int, str, tuple, NO list/dict)
3. Mismo input → mismo output (determinista)

### ❌ MAL: Sin Caché (Ineficiente)

In [ ]:
import time

def fibonacci_slow(n: int) -> int:
    """Calculate Fibonacci number (slow recursive version)."""
    if n < 2:
        return n
    return fibonacci_slow(n - 1) + fibonacci_slow(n - 2)

# This will be very slow for n > 30
start = time.time()
result = fibonacci_slow(35)
end = time.time()
print(f"fibonacci_slow(35) = {result}")
print(f"Time: {end - start:.4f}s")

### ✅ BIEN: Con lru_cache()

In [ ]:
from functools import lru_cache
import time

@lru_cache(maxsize=None)  # Unlimited cache size
def fibonacci_fast(n: int) -> int:
    """
    Calculate Fibonacci number with memoization.
    
    :param n: Position in Fibonacci sequence
    :type n: int
    :return: Fibonacci number at position n
    :rtype: int
    """
    if n < 2:
        return n
    return fibonacci_fast(n - 1) + fibonacci_fast(n - 2)

# This will be extremely fast even for large n
start = time.time()
result = fibonacci_fast(35)
end = time.time()
print(f"fibonacci_fast(35) = {result}")
print(f"Time: {end - start:.6f}s")

# Check cache statistics
print(f"\nCache info: {fibonacci_fast.cache_info()}")

# Can even calculate much larger values
start = time.time()
result = fibonacci_fast(100)
end = time.time()
print(f"\nfibonacci_fast(100) = {result}")
print(f"Time: {end - start:.6f}s")
print(f"Cache info: {fibonacci_fast.cache_info()}")

### Configuración de lru_cache

In [ ]:
from functools import lru_cache

# Limited cache size (LRU eviction when full)
@lru_cache(maxsize=128)
def expensive_computation(x: int, y: int) -> int:
    """
    Simulate expensive computation.
    
    :param x: First parameter
    :type x: int
    :param y: Second parameter
    :type y: int
    :return: Result of computation
    :rtype: int
    """
    print(f"Computing {x} + {y}...")
    return x + y

# First calls compute
print(expensive_computation(1, 2))
print(expensive_computation(3, 4))

# Repeated calls use cache
print(expensive_computation(1, 2))  # No "Computing" message
print(expensive_computation(3, 4))  # No "Computing" message

# Clear cache manually
expensive_computation.cache_clear()
print("\nCache cleared!")
print(expensive_computation(1, 2))  # Computes again

### Ejemplo Práctico: Optimización de Consultas

In [ ]:
from functools import lru_cache
import time

# Simulate database query
@lru_cache(maxsize=100)
def get_user_data(user_id: int) -> dict:
    """
    Fetch user data (simulated with cache).
    
    :param user_id: User ID to fetch
    :type user_id: int
    :return: User data dictionary
    :rtype: dict
    """
    print(f"Fetching user {user_id} from database...")
    time.sleep(0.1)  # Simulate network delay
    return {"id": user_id, "name": f"User{user_id}", "email": f"user{user_id}@example.com"}

# First access - slow
start = time.time()
user = get_user_data(123)
print(f"User: {user}")
print(f"Time: {time.time() - start:.3f}s\n")

# Second access - instant (cached)
start = time.time()
user = get_user_data(123)
print(f"User: {user}")
print(f"Time: {time.time() - start:.6f}s\n")

# Calculate prime numbers with cache
@lru_cache(maxsize=None)
def is_prime(n: int) -> bool:
    """
    Check if number is prime.
    
    :param n: Number to check
    :type n: int
    :return: True if prime
    :rtype: bool
    """
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

# Find primes up to 100
primes = [n for n in range(100) if is_prime(n)]
print(f"Primes up to 100: {primes}")
print(f"Cache info: {is_prime.cache_info()}")

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. Solo para **funciones puras** (sin efectos secundarios, deterministas)
2. Args deben ser **hashables** (int, str, tuple OK; list, dict NO)
3. `maxsize=None` → caché **ilimitado** (cuidado con memoria)
4. `maxsize=128` → default, buen balance memoria/performance
5. Usa `cache_info()` para **estadísticas** y `cache_clear()` para **limpiar**

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Esta función se llama MÚLTIPLES veces con los MISMOS argumentos?"
  - SÍ + cálculo costoso → `lru_cache` es OBLIGATORIO
  - SÍ + cálculo rápido → probablemente no vale la pena
  - NO (args siempre diferentes) → caché no ayuda

- **Pregúntate**: "¿La función es pura?"
  - SÍ (mismo input → mismo output) → seguro usar caché
  - NO (depende de tiempo, random, DB) → NO uses caché

- **Pregúntate**: "¿Cuántas combinaciones de args hay?"
  - Pocas (< 1000) → `maxsize=None` o grande
  - Muchas (> 10000) → `maxsize=128` o 256
  - Infinitas → caché no es apropiado

**Cuándo usar / NO usar**:
- ✅ **Usar cuando**:
  - Función pura con cálculos costosos
  - Recursión (Fibonacci, factorial, paths)
  - Consultas a APIs/DB con mismos params
  - Cálculos matemáticos complejos
  - Validaciones/parseo repetitivo
  
- ❌ **NO usar cuando**:
  - Función con efectos secundarios (print, write file, modify state)
  - Depende de tiempo/random/estado externo
  - Args no hashables (list, dict, set)
  - Función rápida (< 1ms) → overhead del caché no vale la pena
  - Infinitas combinaciones de args → memoria explota

**Ejemplo de cuándo NO usar**:
```python
# ❌ MAL: Función con efectos secundarios
@lru_cache
def get_current_time():
    return datetime.now()  # Siempre retorna mismo valor cacheado!

# ❌ MAL: Args no hashables
@lru_cache
def process_list(items: list):  # list no es hashable
    return sum(items)

# ✅ BIEN: Convierte a tuple
@lru_cache
def process_list(items: tuple):  # tuple es hashable
    return sum(items)
```

**Monitoreo del caché**:
```python
@lru_cache(maxsize=128)
def expensive_func(x):
    return x ** 2

# Después de usar la función
info = expensive_func.cache_info()
print(f"Hits: {info.hits}")      # Veces que encontró en caché
print(f"Misses: {info.misses}")  # Veces que tuvo que calcular
print(f"Hit rate: {info.hits / (info.hits + info.misses):.2%}")
```

**Referencia oficial:** [functools.lru_cache()](https://docs.python.org/3/library/functools.html#functools.lru_cache)

## 6. Combinando Técnicas Funcionales

### Pipeline de Procesamiento de Datos

Las técnicas funcionales brillan cuando se combinan para crear pipelines de transformación de datos elegantes y eficientes.

In [ ]:
from functools import reduce, partial, lru_cache

# Sample data: list of transactions
transactions = [
    {"id": 1, "amount": 100, "type": "credit", "category": "salary"},
    {"id": 2, "amount": 50, "type": "debit", "category": "food"},
    {"id": 3, "amount": 200, "type": "credit", "category": "bonus"},
    {"id": 4, "amount": 30, "type": "debit", "category": "transport"},
    {"id": 5, "amount": 150, "type": "credit", "category": "salary"},
    {"id": 6, "amount": 80, "type": "debit", "category": "food"},
]

# Pipeline: filter credits -> map to amounts -> reduce to sum
is_credit = lambda t: t["type"] == "credit"
get_amount = lambda t: t["amount"]
add = lambda x, y: x + y

total_credits = reduce(
    add,
    map(get_amount, filter(is_credit, transactions)),
    0
)
print(f"Total credits: ${total_credits}")

# More complex pipeline with partial
def filter_by_type(transaction: dict, trans_type: str) -> bool:
    """Filter transactions by type."""
    return transaction["type"] == trans_type

def filter_by_category(transaction: dict, category: str) -> bool:
    """Filter transactions by category."""
    return transaction["category"] == category

# Create specialized filters
is_debit = partial(filter_by_type, trans_type="debit")
is_food = partial(filter_by_category, category="food")

# Calculate total food expenses
food_expenses = reduce(
    add,
    map(get_amount, filter(lambda t: is_debit(t) and is_food(t), transactions)),
    0
)
print(f"Total food expenses: ${food_expenses}")

# Group by category using reduce
def group_by_category(acc: dict, transaction: dict) -> dict:
    """
    Group transactions by category.
    
    :param acc: Accumulator dictionary
    :type acc: dict
    :param transaction: Transaction to add
    :type transaction: dict
    :return: Updated accumulator
    :rtype: dict
    """
    category = transaction["category"]
    if category not in acc:
        acc[category] = []
    acc[category].append(transaction)
    return acc

grouped = reduce(group_by_category, transactions, {})
print(f"\nGrouped by category:")
for category, trans in grouped.items():
    total = sum(t["amount"] for t in trans)
    print(f"  {category}: {len(trans)} transactions, total ${total}")

### Aprendizaje Clave

Combinar `map()`, `filter()`, `reduce()`, `partial()` y `lru_cache()` permite crear pipelines de procesamiento de datos expresivos y eficientes. Cada función tiene un propósito específico, y juntas forman un toolkit poderoso para programación funcional en Python.

**Referencia oficial:** [Functional Programming HOWTO](https://docs.python.org/3/howto/functional.html)

## Ejercicios Prácticos

### 🏋️ Ejercicio 1: Pipeline de Transformación (Básico)

**Objetivo**: Practicar combinación de `filter()`, `map()` y `reduce()`

**Contexto real**: 
Estás procesando datos de ventas. Necesitas calcular el total de ventas de productos caros.

**Instrucciones**:
Dada una lista de números, crea un pipeline que:
1. Filtre solo los números pares
2. Eleve cada número al cuadrado
3. Sume todos los resultados

Usa `filter()`, `map()` y `reduce()`.

**Criterios de éxito**:
- [ ] Usa `filter()` para números pares
- [ ] Usa `map()` para elevar al cuadrado
- [ ] Usa `reduce()` para sumar
- [ ] Resultado correcto: 220

In [ ]:
from functools import reduce

# TODO: Completa el pipeline
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Tu código aquí
result = 0  # Reemplaza con tu solución usando filter, map, reduce

print(f"Result: {result}")  # Should be 220 (2² + 4² + 6² + 8² + 10²)
assert result == 220, f"Expected 220, got {result}"
print("✅ Test pasado!")

<details>
<summary><b>💡 Pista 1</b></summary>

Recuerda que puedes encadenar las operaciones:
```python
result = reduce(suma, map(cuadrado, filter(es_par, numbers)))
```

</details>

<details>
<summary><b>💡 Pista 2</b></summary>

- `filter(lambda x: x % 2 == 0, numbers)` → filtra pares
- `map(lambda x: x ** 2, ...)` → eleva al cuadrado
- `reduce(lambda x, y: x + y, ..., 0)` → suma todo

</details>

<details>
<summary><b>✅ Ver Solución Completa</b></summary>

```python
from functools import reduce

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Solución: Pipeline funcional
result = reduce(
    lambda x, y: x + y,
    map(
        lambda x: x ** 2,
        filter(lambda x: x % 2 == 0, numbers)
    ),
    0
)

print(f"Result: {result}")  # 220
```

**Explicación paso a paso**:
1. `filter(lambda x: x % 2 == 0, numbers)` → [2, 4, 6, 8, 10]
2. `map(lambda x: x ** 2, ...)` → [4, 16, 36, 64, 100]
3. `reduce(lambda x, y: x + y, ..., 0)` → 4 + 16 + 36 + 64 + 100 = 220

**Por qué funciona**:
- Pipeline funcional: cada operación transforma el resultado de la anterior
- Lazy evaluation: filter y map no ejecutan hasta que reduce los consume
- Composición: operaciones se leen de dentro hacia fuera

**Alternativa más legible**:
```python
# Opción 1: Con funciones nombradas
is_even = lambda x: x % 2 == 0
square = lambda x: x ** 2
add = lambda x, y: x + y

result = reduce(add, map(square, filter(is_even, numbers)), 0)

# Opción 2: List comprehension (más Pythonic)
result = sum(x ** 2 for x in numbers if x % 2 == 0)
```

**Conexión con conceptos**:
- **filter()**: Selección de elementos (solo pares)
- **map()**: Transformación uno-a-uno (elevar al cuadrado)
- **reduce()**: Agregación (sumar todos)
- **Pipeline**: Composición de operaciones funcionales

</details>

### 🏋️ Ejercicio 2: Funciones Especializadas con partial() (Intermedio)

**Objetivo**: Practicar creación de funciones especializadas con `partial()`

**Contexto real**: 
Estás construyendo un sistema de pricing para e-commerce. Necesitas calcular precios con diferentes configuraciones de impuestos y descuentos.

**Instrucciones**:
Crea una función `calculate_price()` que tome:
- `base_price`: precio base
- `tax_rate`: tasa de impuesto (decimal)
- `discount`: descuento (decimal)

Luego usa `partial()` para crear:
- `calculate_price_with_vat`: con IVA del 21% (tax_rate=0.21)
- `calculate_price_with_discount`: con 10% de descuento (discount=0.10)
- `calculate_final_price`: con IVA del 21% y 10% de descuento

**Criterios de éxito**:
- [ ] Función `calculate_price` implementada correctamente
- [ ] Usa `partial()` para crear las 3 funciones especializadas
- [ ] Tests pasan

In [ ]:
from functools import partial

# TODO: Implementa la función
def calculate_price(base_price: float, tax_rate: float = 0, discount: float = 0) -> float:
    """
    Calculate final price with tax and discount.
    
    :param base_price: Base price
    :type base_price: float
    :param tax_rate: Tax rate (0.21 = 21%)
    :type tax_rate: float
    :param discount: Discount rate (0.10 = 10%)
    :type discount: float
    :return: Final price
    :rtype: float
    """
    # Tu código aquí
    pass

# TODO: Crea las funciones especializadas usando partial
calculate_price_with_vat = None
calculate_price_with_discount = None
calculate_final_price = None

# Tests
print(f"Base price: $100")
print(f"With VAT (21%): ${calculate_price_with_vat(100)}")
print(f"With discount (10%): ${calculate_price_with_discount(100)}")
print(f"Final price (VAT + discount): ${calculate_final_price(100)}")

assert calculate_price_with_vat(100) == 121.0, "VAT calculation incorrect"
assert calculate_price_with_discount(100) == 90.0, "Discount calculation incorrect"
assert calculate_price_final(100) == 108.9, "Final price calculation incorrect"
print("\n✅ Todos los tests pasaron!")

<details>
<summary><b>💡 Pista 1</b></summary>

La fórmula del precio final es:
```python
price_after_discount = base_price * (1 - discount)
final_price = price_after_discount * (1 + tax_rate)
```

</details>

<details>
<summary><b>💡 Pista 2</b></summary>

Usa `partial()` para fijar los parámetros:
```python
calculate_price_with_vat = partial(calculate_price, tax_rate=0.21)
```

</details>

<details>
<summary><b>✅ Ver Solución Completa</b></summary>

```python
from functools import partial

def calculate_price(base_price: float, tax_rate: float = 0, discount: float = 0) -> float:
    """
    Calculate final price with tax and discount.
    
    Formula: (base_price * (1 - discount)) * (1 + tax_rate)
    """
    price_after_discount = base_price * (1 - discount)
    final_price = price_after_discount * (1 + tax_rate)
    return final_price

# Crear funciones especializadas
calculate_price_with_vat = partial(calculate_price, tax_rate=0.21)
calculate_price_with_discount = partial(calculate_price, discount=0.10)
calculate_final_price = partial(calculate_price, tax_rate=0.21, discount=0.10)

# Pruebas
print(f"Base price: $100")
print(f"With VAT: ${calculate_price_with_vat(100)}")  # 121.0
print(f"With discount: ${calculate_price_with_discount(100)}")  # 90.0
print(f"Final price: ${calculate_final_price(100)}")  # 108.9
```

**Explicación paso a paso**:
1. `calculate_price` aplica descuento primero, luego impuesto
2. `partial()` crea nuevas funciones con parámetros fijos
3. Las funciones especializadas solo necesitan `base_price`

**Por qué funciona**:
- `partial()` "congela" los parámetros especificados
- Cuando llamas `calculate_price_with_vat(100)`, es equivalente a `calculate_price(100, tax_rate=0.21, discount=0)`
- Puedes crear múltiples versiones especializadas sin duplicar código

**Casos de uso reales**:
```python
# Diferentes regiones con diferentes impuestos
price_eu = partial(calculate_price, tax_rate=0.21)  # Europa
price_us = partial(calculate_price, tax_rate=0.08)  # USA
price_uk = partial(calculate_price, tax_rate=0.20)  # UK

# Diferentes tipos de clientes
price_vip = partial(calculate_price, discount=0.20)  # VIP 20% off
price_member = partial(calculate_price, discount=0.10)  # Member 10% off
price_regular = partial(calculate_price, discount=0)  # Regular
```

**Conexión con conceptos**:
- **partial()**: Aplicación parcial de funciones
- **Especialización**: Crear funciones específicas desde genéricas
- **Reutilización**: Una función base, múltiples configuraciones
- **Configuración**: Fijar parámetros de configuración

</details>

### Ejercicio 3: Optimización con lru_cache() (Avanzado)

Implementa una función `count_paths()` que calcule el número de caminos únicos en una cuadrícula de m×n desde la esquina superior izquierda hasta la esquina inferior derecha (solo puedes moverte hacia abajo o hacia la derecha).

1. Primero implementa la versión recursiva sin caché
2. Luego agrega `@lru_cache` y compara el rendimiento
3. Calcula los caminos para una cuadrícula de 20×20

In [ ]:
from functools import lru_cache
import time

# Tu código aquí
def count_paths_slow(m, n):
    """Count paths without cache."""
    pass

@lru_cache(maxsize=None)
def count_paths_fast(m, n):
    """Count paths with cache."""
    pass

# Prueba y compara
print("Testing 10x10 grid:")
start = time.time()
result_slow = count_paths_slow(10, 10)
time_slow = time.time() - start
print(f"Without cache: {result_slow} paths in {time_slow:.4f}s")

start = time.time()
result_fast = count_paths_fast(10, 10)
time_fast = time.time() - start
print(f"With cache: {result_fast} paths in {time_fast:.6f}s")
print(f"Speedup: {time_slow/time_fast:.0f}x faster")

# Now try 20x20 (only with cache!)
print(f"\n20x20 grid: {count_paths_fast(20, 20)} paths")

## Resumen

En este notebook hemos aprendido:

1. **map()**: Aplica una función a cada elemento de un iterable, ideal para transformaciones uniformes de datos

2. **filter()**: Selecciona elementos que cumplen una condición, proporcionando una forma funcional de filtrado

3. **reduce()**: Combina todos los elementos de un iterable en un único valor mediante una función acumulativa

4. **functools.partial()**: Crea funciones especializadas fijando algunos argumentos, mejorando la reutilización de código

5. **functools.lru_cache()**: Implementa memoización automática para optimizar funciones con cálculos costosos

La programación funcional en Python permite escribir código más expresivo, componible y eficiente. Aunque Python no es un lenguaje puramente funcional, estas herramientas son invaluables para procesamiento de datos, optimización de rendimiento, y creación de pipelines de transformación elegantes.

## Preguntas de Autoevaluación

### 1. ¿Cuál es la diferencia principal entre map() y una list comprehension?

**Respuesta:** `map()` devuelve un iterador lazy (no evalúa hasta que se consume), mientras que una list comprehension crea una lista inmediatamente. `map()` es más eficiente en memoria para grandes datasets, pero las list comprehensions suelen ser más legibles y Pythonic para casos simples.

### 2. ¿Por qué reduce() está en functools y no es una función built-in como map() y filter()?

**Respuesta:** Guido van Rossum consideró que `reduce()` es menos legible que alternativas como loops o funciones específicas (`sum()`, `any()`, `all()`). Se movió a `functools` en Python 3 para desalentar su uso excesivo, aunque sigue siendo útil para operaciones de reducción complejas.

### 3. ¿Cuándo usarías partial() en lugar de una lambda?

**Respuesta:** Usa `partial()` cuando necesites: (1) crear funciones reutilizables con nombre, (2) preservar la firma de la función original, (3) trabajar con funciones que esperan callables específicos, o (4) cuando la lambda sería demasiado compleja. `partial()` es más explícito y mantenible.

### 4. ¿Qué significa "LRU" en lru_cache y por qué es importante?

**Respuesta:** LRU significa "Least Recently Used" (Menos Recientemente Usado). Cuando el caché alcanza su tamaño máximo, elimina el elemento que no se ha usado por más tiempo. Esto es importante para controlar el uso de memoria mientras se mantiene en caché los datos más relevantes.

### 5. ¿Qué tipo de funciones son buenas candidatas para lru_cache()?

**Respuesta:** Funciones puras (sin efectos secundarios) con: (1) cálculos costosos, (2) argumentos hashables, (3) llamadas repetidas con los mismos argumentos, y (4) número limitado de combinaciones de argumentos. No uses caché en funciones con efectos secundarios o que dependen de estado externo.

### 6. ¿Cómo combinarías map(), filter() y reduce() para procesar una lista de diccionarios?

**Respuesta:** Primero usa `filter()` para seleccionar diccionarios que cumplan condiciones, luego `map()` para extraer o transformar valores específicos, y finalmente `reduce()` para combinar los resultados en un valor único. Ejemplo: filtrar usuarios activos, mapear a edades, reducir a edad promedio.

### 7. ¿Cuál es la ventaja de usar funciones de orden superior sobre loops tradicionales?

**Respuesta:** Las funciones de orden superior: (1) son más declarativas (expresan QUÉ hacer, no CÓMO), (2) son más componibles (fáciles de combinar), (3) reducen errores (menos estado mutable), (4) son más fáciles de paralelizar, y (5) comunican intención más claramente. Sin embargo, los loops pueden ser más legibles para lógica compleja.

## Recursos y Referencias Oficiales

### Documentación Oficial
- **[Python map() built-in](https://docs.python.org/3/library/functions.html#map)**: https://docs.python.org/3/library/functions.html#map
  - Documentación completa de la función map()

- **[Python filter() built-in](https://docs.python.org/3/library/functions.html#filter)**: https://docs.python.org/3/library/functions.html#filter
  - Documentación completa de la función filter()

- **[functools module](https://docs.python.org/3/library/functools.html)**: https://docs.python.org/3/library/functools.html
  - Módulo completo con reduce(), partial(), lru_cache() y más

- **[functools.reduce()](https://docs.python.org/3/library/functools.html#functools.reduce)**: https://docs.python.org/3/library/functools.html#functools.reduce
  - Documentación específica de reduce()

- **[functools.partial()](https://docs.python.org/3/library/functools.html#functools.partial)**: https://docs.python.org/3/library/functools.html#functools.partial
  - Documentación de aplicación parcial de funciones

- **[functools.lru_cache()](https://docs.python.org/3/library/functools.html#functools.lru_cache)**: https://docs.python.org/3/library/functools.html#functools.lru_cache
  - Documentación del decorador de caché LRU

### Guías y Tutoriales
- **[Functional Programming HOWTO](https://docs.python.org/3/howto/functional.html)**: https://docs.python.org/3/howto/functional.html
  - Guía oficial sobre programación funcional en Python

- **[Python Functional Programming](https://realpython.com/python-functional-programming/)**: https://realpython.com/python-functional-programming/
  - Tutorial completo de Real Python sobre programación funcional

### Mejores Prácticas
- **[PEP 289 - Generator Expressions](https://www.python.org/dev/peps/pep-0289)**: https://www.python.org/dev/peps/pep-0289
  - PEP sobre expresiones generadoras, alternativa a map() y filter()

### Herramientas Relacionadas
- **[itertools module](https://docs.python.org/3/library/itertools.html)**: https://docs.python.org/3/library/itertools.html
  - Módulo con herramientas adicionales para iteradores funcionales

- **[operator module](https://docs.python.org/3/library/operator.html)**: https://docs.python.org/3/library/operator.html
  - Funciones que corresponden a operadores built-in, útiles con map/filter/reduce

### Notas Importantes
- Todos los enlaces están actualizados a partir de 2025
- Se recomienda revisar la documentación oficial regularmente
- Para datasets grandes, considera usar bibliotecas especializadas como pandas o NumPy
- La programación funcional en Python es un complemento, no un reemplazo de otros paradigmas